<a href="https://colab.research.google.com/github/ekaropolus/recommender_and_graph_spp/blob/main/Neo4j_ASDI_Sustainable_Public_Policies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 1.6 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.5-py3-none-any.whl size=123291 sha256=afeac510ca351ca4d73718e300639e16a5ce26b93fd8dd01342189071e5ce807
  Stored in directory: /root/.cache/pip/wheels/73/62/2d/6532ffe9b1fdc65f04e7461279cda78fccaa67cf3c706e6e3d
Successfully built neo4j


In [69]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, person1_name, person2_name):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, person1_name, person2_name)
            for row in result:
                print("Created friendship between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def _create_and_return_friendship(tx, person1_name, person2_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "CREATE (p1:Person { name: $person1_name }) "
            "CREATE (p2:Person { name: $person2_name }) "
            "CREATE (p1)-[:KNOWS]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, person1_name=person1_name, person2_name=person2_name)
        try:
            return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_public_policy(self):
        with self.driver.session(database="neo4j") as session:
            result = session.read_transaction(self._find_and_return_public_policy)
            for row in result:
              print(row)

    @staticmethod
    def _find_and_return_public_policy(tx):
        query = (
            "MATCH (n) "
            "WHERE n.Country = 'MX' "
            "RETURN n.PublicPolicyId as id, n.Level as group limit 10"
        )
        result = tx.run(query)
        return [(row["id"],row["group"]) for row in result]




In [70]:
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
uri = "neo4j+s://2c2e2321.databases.neo4j.io"
user = "neo4j"
password = "kPJiOCU2ekZYW7159DH_2-4-638msHWntxaWbRkxGVQ"
app = App(uri, user, password)


In [71]:
app.find_public_policy()

('AG1', '1')
('AG2', '1')
('AG3', '1')
('AG4', '1')
('AG5', '1')
('BC1', '1')
('BC2', '1')
('BC3', '1')
('BC4', '1')
('BC5', '1')
